## Lending Club Case Study

This project explores how to identify risky loan applicants using Exploratory Data Analysis (EDA). We analyze past loan data to understand factors that influence whether a person will default on a loan.

#### Import Required Libraries

In [15]:
import numpy as np;
import pandas as pd;
import matplotlib.pyplot as plt;
import seaborn as sns;

#### Data Extracting - Load CSV file into data frame

In [26]:
loandf=pd.read_csv('C:\\Users\\RS24050\\Desktop\\loan.csv',low_memory=False);

In [12]:
## Increasing the row size as many of the results are getting hidden ##
## due to large number of rows return as part of compytation. ##

pd.options.display.max_rows = 20000

#### Analyze dataframe head post loading the CSV File

In [ ]:
loandf.head()


In [ ]:
loandf.info()

#### Data Cleaning

##### Check percentage of null values in each coloums in loan datafarme

In [ ]:
loandf.isnull().sum()/len(loandf)*100

##### We can see that there are coloums which are not having any values (100% null values)
##### and high percent of null values:
#####   1. 60%+ [mths_since_last_delinq]
#####  2. 90%+ [mths_since_last_record, next_pymnt_d]
##### We need to drop these coloums from dataframe

In [ ]:

## getting all coloms have 100% null values and droping them 
loandf.dropna(axis=1,how='all',inplace=True)

## droping other coloums with high percentage of null values.
drop_list= ['mths_since_last_delinq','mths_since_last_record', 'next_pymnt_d']
thisFilter = loandf.filter(drop_list)
loandf.drop(thisFilter, inplace=True, axis=1)
loandf.info()

In [37]:
## Droping single value colums
for column in loandf.columns:
    if loandf[column].value_counts().shape[0]==1:
            print("*****"+column+"*******")
            loandf.drop([column],axis=1,inplace=True)

#### Data Manipulation

In [ ]:
## Since we can see that there are more that 25000 distinct employee title so we cannot categorize them into relvent bins
## and thus title wise loan defaulter will not be help full rather we can use employee length for same 
## id, member_id, url,desc,title are not useful for defaulter analysis 
## we can give analysis at state level rather than zip_code level that, that can be more helpful

drop_list= ['id','member_id','url','desc','emp_title','title','zip_code']
thisFilter = loandf.filter(drop_list)
loandf.drop(thisFilter, inplace=True, axis=1)
loandf.info()


In [ ]:
info_which_are_not_available_at_loan_approval_time=['installment',
                                                    'delinq_2yrs',
                                                    'inq_last_6mths',
                                                    'open_acc',
                                                    'total_acc',
                                                    'out_prncp',
                                                    'out_prncp_inv',
                                                    'total_pymnt',
                                                    'total_pymnt_inv',
                                                    'total_rec_prncp',
                                                    'total_rec_int',
                                                    'total_rec_late_fee',
                                                    'recoveries',
                                                    'collection_recovery_fee',
                                                    'last_pymnt_d',
                                                    'last_pymnt_amnt',
                                                    'last_credit_pull_d',                                                    
                                                   ]
thisFilter = loandf.filter(info_which_are_not_available_at_loan_approval_time)
loandf.drop(thisFilter, inplace=True, axis=1)
loandf.info()


In [ ]:
loandf.isnull().sum()/len(loandf)*100

In [ ]:
loandf.emp_length.value_counts()

In [ ]:
loandf.emp_length.fillna(loandf.emp_length.mode()[0],inplace=True)

In [ ]:
loandf.emp_length.value_counts()

In [21]:
## Convert intrest rate to float

In [22]:
loandf["int_rate"]=loandf["int_rate"].apply(lambda x : float(x[0:-1] if x[-1]=='%' else x ))

In [ ]:
loandf["int_rate"]

In [ ]:
loandf.isnull().sum()/len(loandf)*100

In [ ]:
loandf[loandf.revol_util.isna()][['loan_status']].value_counts()

In [ ]:
loandf.shape

In [27]:
loandf.drop(loandf[loandf.revol_util.isna()].index, inplace=True)

In [ ]:
loandf.shape

In [ ]:
len(loandf[(loandf.pub_rec_bankruptcies.isna()) & (loandf.loan_status=="Charged Off")])

In [ ]:
len(loandf[(loandf.pub_rec_bankruptcies.isna()) & (loandf.loan_status!="Charged Off")])

In [ ]:
loandf[(~loandf.pub_rec_bankruptcies.isna()) & (loandf.loan_status=="Charged Off")][['pub_rec_bankruptcies']].value_counts()

In [ ]:
#loandf.drop(['pub_rec_bankruptcies'],axis=1,inplace=True)
loandf[(~loandf.pub_rec_bankruptcies.isna()) & (loandf.loan_status!="Charged Off")][['pub_rec_bankruptcies']].value_counts()

In [ ]:
loandf.pub_rec_bankruptcies.fillna(loandf.pub_rec_bankruptcies.mode()[0],inplace=True)

In [ ]:
loandf.pub_rec_bankruptcies.isna().sum()

In [ ]:
loandf.info()

In [36]:
## Extracting month and year
loandf["issue_month"]=loandf['issue_d'].apply(lambda dt: dt.split("-")[0])     
loandf["issue_year"]=loandf['issue_d'].apply(lambda dt: dt.split("-")[1])  

In [ ]:
loandf.dti.describe()

In [38]:
#[0-5)[5-10)[10,15)[15,20)[20,25)[25,30)
def dticut(dti):
    if dti>=0 and dti<5 :
        return "0-5%"
    elif dti>=5 and dti<10 :
        return "5-10%"
    elif dti>=10 and dti<15:
        return "10-15%"
    elif dti>=15 and dti<20:
        return "15-20%"
    elif dti>=20 and dti<25:
        return "20-25%"
    elif dti>=25 and dti<30:
        return "25-30%"
    else:
        return "30+%"
loandf['dti_group']=loandf.dti.apply(lambda x: dticut(x))

In [39]:
#[0-5)[5-10)[10,15)[15,20)[20,25)
def intrest_rate_cut(int_rate):
    if int_rate>=0 and int_rate<5 :
        return "0-5%"
    elif int_rate>=5 and int_rate<10 :
        return "5-10%"
    elif int_rate>=10 and int_rate<15:
        return "10-15%"
    elif int_rate>=15 and int_rate<20:
        return "15-20%"
    else:
        return "20-25%"   
loandf['intrest_rate_group']=loandf.int_rate.apply(lambda x: intrest_rate_cut(x))

In [ ]:
sns.boxplot(y=loandf.annual_inc)

In [ ]:
loandf.annual_inc.describe()

In [ ]:
np.percentile(loandf.annual_inc,[25,75,50])

In [ ]:
qutile=np.percentile(loandf.annual_inc,[25,75])
iqr=qutile[1]-qutile[0]
iqr
hb=qutile[1]+( iqr*1.5)
loandf[(loandf.annual_inc>hb)][['annual_inc','loan_status']].sort_values(by="annual_inc",ascending=False)
loandf[(loandf.annual_inc>hb)].shape

In [44]:
loandf.drop(loandf[(loandf.annual_inc>hb)].index,inplace=True)

In [ ]:
loandf[(loandf.annual_inc>hb)].shape

In [ ]:
hb

In [ ]:
sns.boxplot(y=loandf.annual_inc)

In [ ]:
sns.boxplot(y=loandf.loan_amnt)

In [ ]:
loandf[loandf.loan_amnt>32000].shape

In [ ]:
sns.boxplot(y=loandf.int_rate)

In [ ]:
qutile=np.percentile(loandf.int_rate,[25,75])
iqr=qutile[1]-qutile[0]
iqr
hb=qutile[1]+( iqr*1.5)
hb

In [52]:
loandf.drop(loandf[(loandf.int_rate>hb)].index,inplace=True)

In [ ]:
sns.boxplot(loandf.dti)

In [ ]:
sns.countplot(x=loandf.term,hue=loandf.loan_status)

In [ ]:
sns.countplot(x=loandf.grade,hue=loandf.loan_status,order=loandf.grade.value_counts().index.sort_values())

In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=True)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.sub_grade,hue=loandf.loan_status,order=loandf.sub_grade.value_counts().index.sort_values(),ax=ax[0])

ch=loandf[loandf.loan_status=="Charged Off"]
sns.countplot(x=ch.sub_grade,order=sorted(ch.sub_grade.unique()),ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.countplot(x=loandf.emp_length,order=['< 1 year','1 year','2 years','3 years','4 years','5 years','6 years','7 years','8 years','9 years','10+ years'])

In [ ]:
loandf.home_ownership.value_counts()

In [59]:
# None is not a valid values for home ownership so we can  assign the equivalnet valid values which is OTHER

In [60]:
loandf["home_ownership"]=loandf.home_ownership.apply(lambda x : "OTHER" if x=="NONE" else x)

In [ ]:
sns.countplot(x=loandf.home_ownership)

In [ ]:
sns.countplot(x=loandf.verification_status)

In [ ]:
sns.countplot(x=loandf.issue_year,hue=loandf.loan_status)

In [ ]:
sns.countplot(x=loandf.purpose,hue=loandf.loan_status)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.countplot(x=loandf.addr_state)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.countplot(x=loandf.pub_rec,hue=loandf.loan_status)
plt.show()

In [ ]:
sns.countplot(x=loandf.pub_rec_bankruptcies,hue=loandf.loan_status)
plt.show()

## BiVariat Analysis

In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.dti_group,hue=loandf.loan_status,order=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%"],ax=ax[0])

o=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%"]
per=[round((len(loandf[(loandf.dti_group==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.dti_group==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()


In [69]:
# we can se that as chances of getting default increase as the dti ratio increases.


In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.issue_year,hue=loandf.loan_status,ax=ax[0])

o=['11','10','09','08','07']
per=[round((len(loandf[(loandf.issue_year==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.issue_year==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [71]:
# We can see that major loans we taken in 2011 and 2010 but most of the relative
# loan defaulter  were there in 2007 and 2008 that is pre kicking of 2008 crices 
# and then again in 2011

In [ ]:
sns.countplot(x=loandf.issue_year,hue=loandf.purpose)

In [73]:
#from 2011 to 2008  major loans were taken for debt consolidation


In [ ]:
ch=loandf[loandf.loan_status=="Charged Off"]
sns.countplot(x=ch.purpose)
plt.xticks(rotation=90)
plt.show()

In [75]:
# loan takem for debt consolidation has high chances for defaulting.


In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
    
ch=loandf[loandf.loan_status=="Charged Off"]
sns.countplot(x=ch.purpose,ax=ax[0])

o=loandf.purpose.value_counts().index
per=[round((len(loandf[(loandf.purpose==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.purpose==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:
o=loandf.purpose.value_counts().index
per=[{oo,round((len(loandf[(loandf.purpose==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.purpose==oo)]),2)*100} for oo in o]
per

In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
ch=loandf[loandf.loan_status=="Charged Off"]
sns.countplot(x=ch.grade,order=['A','B','C','D','E','F','G'],ax=ax[0])

o=loandf.grade.value_counts().index
per=[round((len(loandf[(loandf.grade==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.grade==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()



In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
ch=loandf[loandf.loan_status=="Charged Off"]
sns.countplot(x=ch.sub_grade,order=sorted(ch.sub_grade.unique()),ax=ax[0])

o=loandf.sub_grade.value_counts().index
per=[round((len(loandf[(loandf.sub_grade==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.sub_grade==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:

plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.home_ownership,hue=loandf.loan_status,ax=ax[0])

o=loandf.home_ownership.value_counts().index
per=[round((len(loandf[(loandf.home_ownership==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.home_ownership==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:

plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.verification_status,hue=loandf.loan_status,ax=ax[0])

o=loandf.verification_status.value_counts().index
per=[round((len(loandf[(loandf.verification_status==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.verification_status==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:
ch=loandf[(loandf.loan_status=="Charged Off") & (loandf.verification_status=="Verified")]
sns.countplot(x=ch.home_ownership,hue=loandf.grade)

In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.term,hue=loandf.loan_status,ax=ax[0])

o=loandf.term.value_counts().index
per=[round((len(loandf[(loandf.term==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.term==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:
loandf.info()

In [ ]:
loandf.int_rate.describe()

In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.intrest_rate_group,hue=loandf.loan_status,order=["0-5%","5-10%","10-15%","15-20%","20-25%"],ax=ax[0])

o=["5-10%","10-15%","15-20%","20-25%"]
per=[round((len(loandf[(loandf.intrest_rate_group==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.intrest_rate_group==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()


In [ ]:
plt.rcParams["figure.figsize"] = [10.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.emp_length,hue=loandf.loan_status,ax=ax[0])

o=loandf.emp_length.value_counts().index
per=[round((len(loandf[(loandf.emp_length==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.emp_length==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [16.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.addr_state,hue=loandf.loan_status,ax=ax[0])

o=loandf.addr_state.value_counts().index
per=[round((len(loandf[(loandf.addr_state==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.addr_state==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [16.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.pub_rec_bankruptcies,hue=loandf.loan_status,ax=ax[0])

o=loandf.pub_rec_bankruptcies .value_counts().index
per=[round((len(loandf[(loandf.pub_rec_bankruptcies ==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.pub_rec_bankruptcies ==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [16.00, 6]
plt.rcParams["figure.autolayout"] = True
fig,ax=plt.subplots(ncols=2,nrows=1,sharex=True,sharey=False)
for tick in ax[0].get_xticklabels():
    tick.set_rotation(90)
for tick in ax[1].get_xticklabels():
    tick.set_rotation(90)
sns.countplot(x=loandf.pub_rec,hue=loandf.loan_status,ax=ax[0])

o=loandf.pub_rec.value_counts().index
per=[round((len(loandf[(loandf.pub_rec ==oo) & (loandf.loan_status=="Charged Off")]))/
    len(loandf[(loandf.pub_rec==oo)]),2)*100 for oo in o]
ax[1].set_ylabel('Percent%')
sns.barplot(x=o,y=per,ax=ax[1])

plt.xticks(rotation=90)
plt.show()

In [ ]:
ch=loandf[loandf.loan_status == "Charged Off"]
sns.countplot(x=ch.dti_group,hue=ch.purpose,order=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%"])
plt.show()
ot=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%"]
op=loandf.purpose.value_counts().index
for o_ot in ot:
    pp=ch[(ch.dti_group == o_ot)]
    per=[round(len(pp[(pp.purpose == p_op)])/len(loandf[(loandf.dti_group == o_ot)& (loandf.purpose == p_op)]),2)*100 for p_op in op]
    plt.title(o_ot)
    sns.barplot(x=op,y=per)
    plt.xticks(rotation=90)
    plt.show()

      


In [ ]:
ch=loandf[loandf.loan_status == "Charged Off"]

ot=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%"]
op=loandf.term.value_counts().index
for o_ot in ot:
    pp=ch[(ch.dti_group == o_ot)]
    per=[round(len(pp[(pp.term == p_op)])/len(loandf[(loandf.dti_group == o_ot)& (loandf.term == p_op)]),2)*100 for p_op in op]
    plt.title(o_ot)
    sns.barplot(x=op,y=per)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
ch=loandf[loandf.loan_status == "Charged Off"]

ot=["0-5%","5-10%","10-15%","15-20%","20-25%","25-30%"]
op=loandf.emp_length.value_counts().index
for o_ot in ot:
    pp=ch[(ch.dti_group == o_ot)]
    per=[round(len(pp[(pp.emp_length == p_op)])/len(loandf[(loandf.dti_group == o_ot)& (loandf.emp_length == p_op)]),2)*100 for p_op in op]
    plt.title(o_ot)
    sns.barplot(x=op,y=per)
    plt.xticks(rotation=90)
    plt.show()